In [116]:
from multibind.nonequilibrium import rate_matrix
import multibind as mb
import numpy as np
from equil import run

np.set_printoptions(precision=3)

In [117]:
def build_base_multibind_model(input_rates : str, pH : float = 8, Na : float = 0.1, verbose : bool = False):

    c, k, std = rate_matrix(input_rates)

    states = c.states.values
    free_energies = c.g_mle
    g_std_err = c.std_errors

    new_graph = c.graph.copy()

    for index, data in new_graph.iterrows():
        state1, state2, value, variance, ligand, std = data
        if verbose:
            print(state1, state2, value, variance, ligand, std)

        if (state1[-1] == "H" and state2[-1] == "0") or (state1[-1] == "A" and state2[-1] == "0"):
            # backwards proton reaction
            new_graph.at[index, 'state1'] = state2
            new_graph.at[index, 'state2'] = state1
            new_graph.at[index, 'value'] = -value

            value = new_graph.value[index]
            state1 = new_graph.state1[index]
            state2 = new_graph.state2[index]

        if state1[-1] == "0" and state2[-1] == "H":
            new_graph.at[index, 'ligand'] = "H+"
            new_graph.at[index, 'value'] = dG2pKa(new_graph.value[index], pH)
            new_graph.at[index, 'variance'] = new_graph.variance[index] / np.log(10)**2
        if state1[-1] == "0" and state2[-1] == "A":
            new_graph.at[index, 'ligand'] = "Na+"
            new_graph.at[index, 'value'] = new_graph.value[index] + np.log(Na)

    c_equil = mb.Multibind()
    c_equil.graph = new_graph
    c_equil.states = c.states
    
    return c_equil, k, pH, Na


def dG2pKa(dG : float, pH : float = 0.0) -> float:
    '''Calculate the pKa from the free energy difference.
    Defaults to standard state (pH = 0).
    '''
    return pH - dG / np.log(10)

In [118]:
def report_results(input_rates, pH, Na, title=None):
    if title:
        print(f"========== {title} ==========")
    c, k, _, _ = build_base_multibind_model(input_rates, pH=pH, Na=Na)
    c.concentrations = {"Na+": 0.100}
    c.build_cycle(pH=8)
    c.MLE()
    scanner = mb.multibind.MultibindScanner(statefile=None, graphfile=None)
    scanner.c = c
    sod_bound = {'IF0': 'unbound',
             'IFH': 'unbound',
             'IFNA': 'bound',
             'OF0': 'unbound',
             'OFH': 'unbound',
             'OFNA': 'bound',
            }

    prot_bound = {'IF0': 'unbound',
                  'IFH': 'bound',
                  'IFNA': 'unbound',
                  'OF0': 'unbound',
                  'OFH': 'bound',
                  'OFNA': 'unbound',
                 }

    conf = {'IF0': 'inward',
            'IFH': 'inward',
            'IFNA': 'inward',
            'OF0': 'outward',
            'OFH': 'outward',
            'OFNA': 'outward',
           }

    scanner.c.states['conf'] = list(map(lambda x: conf[x[0]], scanner.c.states.values))
    scanner.c.states['prot_bound'] = list(map(lambda x: prot_bound[x[0]], scanner.c.states.values))
    scanner.c.states['sod_bound'] = list(map(lambda x: sod_bound[x[0]], scanner.c.states.values))
    run(scanner)
    return c.graph

In [119]:
report_results("inputs/diffusion_rates.csv", pH=8, Na=0.1, title="Realistic")

========== Realistic ==========
IFH (0.0) --> IF0 (-3.584631285478887) => -3.584631285478887 ± 0.09519369164929858
IF0 (-3.584631285478887) --> IFNA (-4.995919960795005) => -1.411288675316118 ± 0.09360123218286136
IFNA (-4.995919960795005) --> OFNA (-4.523709030956624) => 0.472210929838381 ± 0.06624678370582313
OFNA (-4.523709030956624) --> OF0 (-3.883497606097211) => 0.6402114248594133 ± 0.06359939942070127
OF0 (-3.883497606097211) --> OFH (-0.4720897659961597) => 3.411407840101051 ± 0.06600233311482191
OFH (-0.4720897659961597) --> IFH (0.0) => 0.4720897659961597 ± 0.07071569122170802
-1.1102230246251565e-16


,state1,state2,value,variance,ligand,standard_state
0,IF0,IFH,6.397940,0.003419,H+,1
1,IF0,IFNA,-3.822411,0.018447,Na+,1
2,IFNA,OFNA,0.470004,0.000556,helm,1
3,OF0,OFNA,-2.935108,0.003488,Na+,1
4,OF0,OFH,6.522879,0.001113,H+,1
5,OFH,IFH,0.470004,0.000556,helm,1


<Figure size 300x200 with 0 Axes>

<Figure size 300x200 with 0 Axes>

<Figure size 300x200 with 0 Axes>

In [120]:
report_results("inputs/diffusion_rates_scaled.csv", pH=0, Na=1, title="STD STATE")

========== STD STATE ==========
IFH (0.0) --> IF0 (-3.666425192056425) => -3.666425192056425 ± 0.05096349906596548
IF0 (-3.666425192056425) --> IFNA (-5.009601361096364) => -1.3431761690399395 ± 0.0593921421755263
IFNA (-5.009601361096364) --> OFNA (-4.535045106035219) => 0.4745562550611453 ± 0.05533607405298931
OFNA (-4.535045106035219) --> OF0 (-3.8778569619085586) => 0.6571881441266605 ± 0.044790886513047024
OF0 (-3.8778569619085586) --> OFH (-0.4745588120478971) => 3.4032981498606616 ± 0.0341717537142171
OFH (-0.4745588120478971) --> IFH (0.0) => 0.4745588120478971 ± 0.035964121200539635
1.6653345369377348e-16


,state1,state2,value,variance,ligand,standard_state
0,IF0,IFH,6.397940,0.000472,H+,1
1,IF0,IFNA,-3.822411,0.018419,Na+,1
2,IFNA,OFNA,0.470004,0.000556,helm,1
3,OF0,OFNA,-2.935108,0.003461,Na+,1
4,OF0,OFH,6.522879,0.000052,H+,1
5,OFH,IFH,0.470004,0.000556,helm,1


<Figure size 300x200 with 0 Axes>

<Figure size 300x200 with 0 Axes>

<Figure size 300x200 with 0 Axes>